#### Generator

In [8]:
''' 'for' loop retrieve and process elements 'one at a time'.
  Ex. A file may be read a line one at a time.

'comprehension' retrieve, process and collect elements 'as a whole'.
  Ex. A file can be read and returned as a list of all lines.

'one at a time' uses less memory but 'as a whole' is more efficent.
Python allows both but 'lazy' is more favorable.

A 'generator' is a function(object) that returns elements one at a time.
When a function returns a value, its execution terminates.
But when a generator returns a value, it does not terminate.
As the caller calls next(<generator>), it returns a next value.
next() is a builtin function.
The process may continue until no more values, a StopIteration error will thrown.
Ex. range() is an iterable (not a generator),
    iter(<iterable>) returns a generator which can be used to access the value. '''
r = range(3)        ## range
g = iter(r)         ## generator
print(next(g), next(g), next(g))
# next(g)          ## StopIteration error
## When a generator returns the last value, it is done and cannot be used again.

0 1 2


In [10]:
''' In case we do not know how many values. '''
g = iter(r)         ## generator
try:
    while True:
        print(next(g), end=', ')
except StopIteration:
    print('Stop')           ## 0, 1, 2, Stop

''' Python simplify iterating range and generator with 'for' loop and comprehension. '''
for i in r:              ## Try:  iter(r):
    print(i, end=', ')       ## 0, 1, 2,

0, 1, 2, Stop
0, 1, 2, 

In [11]:
''' A functions that 'yield' a value is called 'function generator' or just generator.
A generator yields values without destroy its namespace, just suspend it.
When next() the generator will resume from where it was suspended.
The suspend/resume may continue unitl no more statements to continue, then 
a StopIteration error isl thrown. There is no ways to check before hand.
Ex. Create our generator that as a range. '''
def my_generator(start, stop, step=1):
    i = start
    while i < stop:
        yield i
        i += step

print([i for i in my_generator(0, 3)])

[0, 1, 2]


#### Comprehension Generator

In [13]:
''' Comprehension that surrounded with [ ] returns a list.
                       surrounded with { } returns a set or a dict.
                   but surrounded with ( ) returns a generator.
Instead of creating a generator with function, we can create a generator with comprehension. '''
g = (x for x in range(3))
# print(type(g))          ##  <class 'generator'>
print([x for x in g])   ##  [0, 1, 2]

[0, 1, 2]


In [15]:
''' A comprehension generator is more compact but when it is done it cannot be used
in the next round of nested iterations. '''
g = (i for i in [0, 1])
print([(x, y) for x in g
              for y in g])	 ##  [(0, 1)]

''' Function generators are more convenient in nested iteration. '''
print([(x, y) for x in my_generator(0, 2)
              for y in my_generator(0, 2)])
                        ##  [(0, 0), (0, 1), (1, 0), (1, 1)]

[(0, 1)]
[(0, 0), (0, 1), (1, 0), (1, 1)]


#### Lazy VS Eager

In [16]:
def gen():
    print('one', end=',')
    yield 1
    print('two', end=',')
    yield 2

''' 'for' loops lazyly creates an element and process one at a time. '''
for i in gen():
    print(i, end=',')       ##  one,1,two,2,
print()

''' Comprehension eagerly creates all elements and returns as a whole. '''
c = [i for i in gen()]      ##  one,two
print(c)                    ##  [1, 2]

one,1,two,2,
one,two,[1, 2]


#### yield from

In [19]:
''' To reduce the overhead of supend/resume a generator we can move the generator into a yield.
Python provides 'yield from <generator>' which is the same as:
            for x in <generator>:
                yield x 
'yield from' simplifies creating generator from a generator (or range() and collection).'''
def gen():
    yield from (x for x in range(3))   ## Try: yield from range(3)

print([x for x in gen()]) 

[0, 1, 2]


#### Stateful Function
A stateful function can remember its state between calls.

In [22]:
''' Ex.Factorial:
           fac(n) = 1*2*3*...(n-1)*n
Create a list of 10 factorials.

    1. Simple factorial function starts from 1 every calls. '''
def fac(n):      ## 'n' is the number of factorials to be created.
    r = 1
    for i in range(2, n+1):
        r *= i
    return r

''' 2. Factorial Generator uses result from the previous round. '''
def fac_gen(n):   
    r, i = 1, 0
    while i < n:
        i += 1
        yield r
        r *= i

print([fac(n) for n in range(10)])
print([f for f in fac_gen(10)])

[1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880]
[1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880]


In [23]:
''' Ex. To generate a sequence of 10 non repeated random numbers.  '''
import random
def rand_gen(n):
    ''' The generator may keep a set of already generated numbers. '''
    s = set()
    while len(s) < n:
        r = random.randint(1, n)
        if r not in s:
            s.add(r)
            yield r

print([n for n in rand_gen(10)])

[8, 7, 9, 10, 5, 2, 4, 1, 6, 3]


### Coroutine

In [ ]:
''' A 'coroutine' is a function with
               <variable> = yield <expression>

When a coroutine function is called, it suspends the execution.
Until when the caller call next(<coroutine>), the coroutine resumes and 
proceeds to the first 'yield', this is called 'priming', and returns 
the value of <expression> then waits.
The caller receives the yielded value and process.
Then when the caller calls <coroutine>.send(<value>), the coroutine resumes 
and the sent <value> is assigned to the <variable> that is waiting on the left
of assignment, then processes to next statement if any.
If there is no next 'yield', a StopIterationwill will thrown. '''

In [24]:
def my_coroutine():
    print('start')
    x = yield 1
    print('x = ', x)
    y = yield 2
    print('y = ', y)

try:
    c = my_coroutine()         ## No execution yet.
    print(next(c))             ## 'start'    proceeds to the first 'yield'
                               ## 1
    print(c.send('hello'))     ## x =  hello
                               ## 2
    print(c.send('hi'))        ## y =  hi
except StopIteration as e:
    print('Stop')              ## 'Stop'

start
1
x =  hello
2
y =  hi
Stop


In [25]:
''' A coroutine may be stopped before elements run-out, by the caller calls close(<coroutine>). '''
c = my_coroutine()
print(next(c))              #  start
                            #  1
print(c.send('hello'))      #  x =  hello
                            #  2
c.close()

start
1
x =  hello
2


In [ ]:
''' A coroutine may be stopped before elements run-out,
  when the caller calls close() on its object. '''
def close_test():
    c = my_coroutine()
    print(next(c))              #  start
                                #  1
    print(c.send('hello'))      #  x =  hello
                                #  2
    c.close()

In [27]:
''' Generators can send data to the caller only.
    Coroutines may send data both ways.

An infinite coroutine is implemented by yielding in an infinite loop.
A coroutine may stop itself or a stop command from the caller.
Ex. An infinite sequence coroutine. '''
def inc(first=0):
    n = first
    while True:
        cmd = yield n
        if cmd == 'stop':
            break
        n += 1
    yield None

s = inc()
i = next(s)             ##  Priming.
while True:
    if i >= 10:
        s.send('stop')
        break
    print(i, end=',')
    i = s.send('more')

0,1,2,3,4,5,6,7,8,9,